In [2]:
import pandas as pd
import matplotlib as plot
import nltk 
from scipy.sparse import csr_matrix, hstack,vstack
import numpy as np

In [2]:
%%time
#train read
train=pd.read_table('../input/train.tsv')

#train=train.head(1000)

In [ ]:
#global
frases=pd.DataFrame()


In [3]:
%%time
test=pd.read_table('../input/test.tsv')
train['item_description_upper']= train['item_description'].str.upper()
test['item_description_upper']= test['item_description'].str.upper()
txt=train['item_description_upper'].str.cat(sep='.')+test['item_description_upper'].str.cat(sep='.')
len(txt),len(train['item_description_upper'])
del test

Wall time: 2.86 s


In [4]:
%%time
sent=nltk.sent_tokenize(txt)

Wall time: 1min 45s


In [5]:
len(sent)

3533120

In [6]:
fdist_sent = nltk.FreqDist(sent)

In [7]:
len(fdist_sent)

2443529

In [8]:
frases=pd.DataFrame(fdist_sent.most_common(len(fdist_sent)),columns=['Word', 'Frequency'])

In [9]:
frases['len']=frases['Word'].str.len()

In [10]:
frases.head()

,Word,Frequency,len
0,NEVER WORN.,6370,11
1,PRICE IS FIRM.,5347,14
2,FREE SHIPPING!,4938,14
3,SIZE SMALL.,4599,11
4,GREAT CONDITION.,4579,16


In [11]:
frases=frases[frases['len']>2]

In [12]:
len(frases)

2443478

In [13]:
frases=frases[frases['Frequency']>5]
frases=frases.head(200)

In [14]:
len(frases)

200

In [15]:
frases.head(10)

,Word,Frequency,len
0,NEVER WORN.,6370,11
1,PRICE IS FIRM.,5347,14
2,FREE SHIPPING!,4938,14
3,SIZE SMALL.,4599,11
4,GREAT CONDITION.,4579,16
5,BRAND NEW.,4436,10
6,SIZE MEDIUM.,4155,12
7,NEVER USED.,3850,11
8,EXCELLENT CONDITION.,3654,20
9,WORN ONCE.,3493,10


In [1]:
X_train = csr_matrix((len(train),0), dtype=int)
X_train.shape

NameError: name 'csr_matrix' is not defined

In [17]:
len(train)

1482535

In [18]:
%%time
train["tokenized_sents"] = train["item_description_upper"].fillna("").map(nltk.sent_tokenize)

Wall time: 1min 46s


In [19]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,item_description_upper,tokenized_sents
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,NO DESCRIPTION YET,[NO DESCRIPTION YET]
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,THIS KEYBOARD IS IN GREAT CONDITION AND WORKS ...,[THIS KEYBOARD IS IN GREAT CONDITION AND WORKS...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,ADORABLE TOP WITH A HINT OF LACE AND A KEY HOL...,[ADORABLE TOP WITH A HINT OF LACE AND A KEY HO...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,NEW WITH TAGS. LEATHER HORSES. RETAIL FOR [RM]...,"[NEW WITH TAGS., LEATHER HORSES., RETAIL FOR [..."
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,COMPLETE WITH CERTIFICATE OF AUTHENTICITY,[COMPLETE WITH CERTIFICATE OF AUTHENTICITY]


In [20]:
def index_va(sentence):
    try:
        return train[train['tokenized_sents'].apply(lambda x: sentence in x)].index.values.tolist()
    except:
        []

In [21]:
%%time
frases['index_va']=frases["Word"].apply(index_va)

Wall time: 1min 58s


In [22]:
frases.head()

,Word,Frequency,len,index_va
0,NEVER WORN.,6370,11,"[284, 585, 597, 842, 1026, 1076, 1231, 1370, 1..."
1,PRICE IS FIRM.,5347,14,"[202, 548, 551, 716, 757, 830, 905, 1003, 1410..."
2,FREE SHIPPING!,4938,14,"[163, 603, 815, 910, 1060, 1273, 1599, 2134, 2..."
3,SIZE SMALL.,4599,11,"[53, 662, 907, 1026, 1142, 1343, 1549, 2431, 2..."
4,GREAT CONDITION.,4579,16,"[359, 686, 841, 1116, 1386, 1459, 1538, 1755, ..."


In [23]:
longi=len(train)
print(X_train.shape)
for index, row in frases.iterrows():
#    X_train=hstack([X_train,csr_matrix(([1 for i  in range(len(row['index_va']))],(row['index_va'],[0])))])
    unos=[1 for i  in range(len(row['index_va']))]
    filas=row['index_va']
    columnas=[0 for i  in range(len(row['index_va']))]
#    print(len(unos),len(filas),len(columnas))
    Y=csr_matrix((unos,(filas,columnas)),shape=(longi,1))
    X_train=hstack((X_train,Y))
print(X_train.shape,Y.shape)    

(1482535, 0)
(1482535, 200) (1482535, 1)


In [ ]:
def vectorize_sentence()
